# Project Assignment: Short Video Recommender System (KuaiRec)

Dataset Source: [Kuairec](https://kuairec.com/)

Arxiv Paper: [KuaiRec: A Fully-observed Dataset and Insights for Evaluating Recommender Systems](https://arxiv.org/pdf/2202.10842)

## Dataset import

The server is down, please download from the Google Drive in the given link.

In [ ]:
!wget https://nas.chongminggao.top:4430/datasets/KuaiRec.zip --no-check-certificate
!unzip KuaiRec.zip

## Imports

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


# I get my dataset from a Kaggle input
DATA_PATH = "/kaggle/input/kuairec/KuaiRec 2.0/data"
if not os.path.exists(DATA_PATH):
   DATA_PATH = f"{os.getcwd()}/KuaiRec/data"
if not os.path.exists(DATA_PATH):
   DATA_PATH = f"{os.getcwd()}/KuaiRec 2.0/data"
if not os.path.exists(DATA_PATH):
   raise FileNotFoundError("KuaiRec dataset not found. Please check the path.")

DATA_PATH

2025-04-22 21:58:16.402273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745351896.488711   87517 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745351896.513417   87517 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745351896.711632   87517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745351896.711661   87517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745351896.711663   87517 computation_placer.cc:177] computation placer alr

'/home/tofeha/ING2/ING2/REMA1/FinalProject_2025_aziz.zeghal/KuaiRec 2.0/data'

# Step 1: Load the dataset

In [2]:
def data_clear(df : pd.DataFrame) -> pd.DataFrame:
    # Date is time in a weird format

    # Time and Date are duplicated of timestamp, we can drop them
    df.drop(columns=["time", "date"], inplace=True)
    # Not a problem, we want to keep the data for the density
    df = df.astype({
        "user_id": "int32",
        "video_id": "int32",
        "play_duration":"int32",
        "timestamp": "int64",
        "watch_ratio": "float32"}, errors="ignore")
    
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    df = df[df["timestamp"] >= 0]
    
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    return df

In [ ]:
def my_describe(df : pd.DataFrame) -> pd.DataFrame:
    """
    Custom describe for datasets containing user_id and video_id
    """
    print(f"Shape of the small matrix: {df.shape}")
    unique_users = df["user_id"].nunique()
    unique_posts = df["video_id"].nunique()
    print(f"Number of unique users: {unique_users}")
    print(f"Number of unique posts: {unique_posts}")
    print(f"Matrix sparsity: {len(df) /(unique_posts * unique_users) * 100}%")
    return df.describe()

## Small matrix

This table has a density of 99.6%. This means that 99.6% of the entries in the matrix are non-zero, indicating that most users have interacted with most items.

In [ ]:
small_matrix = pd.read_csv(f"{DATA_PATH}/small_matrix.csv")

small_matrix = data_clear(small_matrix)


## Big matrix

This table has a density of 16.3%. We will use this matrix for our training and testing.

It contains more interactions with the same users/items of the small matrix. We do not need to substract the small matrix.

In [10]:
big_matrix = pd.read_csv(f"{DATA_PATH}/big_matrix.csv")

big_matrix = data_clear(big_matrix)


In [14]:
big_matrix

,user_id,video_id,play_duration,video_duration,timestamp,watch_ratio
0,0,3649,13838,10867,2020-07-04 16:08:23,1.273396
1,0,9598,13665,10984,2020-07-04 16:13:41,1.244082
2,0,5262,851,7908,2020-07-04 16:16:06,0.107613
3,0,1963,862,9590,2020-07-04 16:20:26,0.089885
4,0,8234,858,11000,2020-07-04 16:43:05,0.078000
...,...,...,...,...,...,...
12530801,7175,1281,34618,140017,2020-09-05 07:07:10,0.247241
12530802,7175,3407,12619,21888,2020-09-05 07:08:45,0.576526
12530803,7175,10360,2407,7067,2020-09-05 11:10:29,0.340597
12530804,7175,10360,6455,7067,2020-09-05 11:10:36,0.913400


## Misc

In [ ]:
print(f"Proportion of small_matrix relative to big_matrix: {small_matrix.shape[0] * 100 / big_matrix.shape[0]:.2f}%")

## Item category encoding

We have the caracteristics of the videos (author_id, video_type...) but this part requires less preprocessing.

For Content-based filtering, we need to use features of the videos (list of tags). No need for TF-IDF, we will use a simple one-hot encoding.

In [4]:
# No missing values for this data
item_categories = pd.read_csv(f"{DATA_PATH}/item_categories.csv")

# Transform the feat column to a list (evaluate with python)
item_categories["feat"] = item_categories["feat"].apply(eval)

## Item daily features

This dataset is also interesting for content-based filtering.

Mostly composed of textual data, we will use a TF-IDF vectorizer to encode the features of the videos.

In [19]:
item_daily_features = pd.read_csv(f"{DATA_PATH}/item_daily_features.csv", lineterminator='\n')
item_daily_features

,video_id,date,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,video_height,...,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
0,0,20200705,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,8,8,0,0,3,3,NaN,NaN,NaN,NaN
1,0,20200706,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,5,5,NaN,NaN,NaN,NaN
2,0,20200707,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,0,0,NaN,NaN,NaN,NaN
3,0,20200708,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,3,3,0,0,3,3,NaN,NaN,NaN,NaN
4,0,20200709,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,2,1,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343336,10723,20200905,236,NORMAL,2020-09-05,ShortImport,public,4833.0,720,1280,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
343337,10724,20200905,5271,NORMAL,2020-09-05,LongImport,public,54720.0,720,1280,...,1,1,0,0,0,0,0.0,0.0,0.0,0.0
343338,10725,20200905,1924,NORMAL,2020-09-05,ShortImport,public,15800.0,576,1024,...,5,5,0,0,4,4,0.0,0.0,0.0,0.0
343339,10726,20200905,7604,NORMAL,2020-09-05,ShortImport,public,5132.0,528,960,...,2,2,0,0,1,1,0.0,0.0,0.0,0.0


## Caption Category

In [ ]:
caption_category = pd.read_csv(f"{DATA_PATH}/kuairec_caption_category.csv", lineterminator='\n')
caption_category

## User features

In [5]:
user_features = pd.read_csv(f"{DATA_PATH}/user_features.csv", lineterminator='\n')
user_features

,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,follow_user_num_range,fans_user_num,fans_user_num_range,friend_user_num,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,high_active,0,0,0,5,"(0,10]",0,0,0,...,184,6,3,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,full_active,0,0,0,386,"(250,500]",4,"[1,10)",2,...,186,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,full_active,0,0,0,27,"(10,50]",0,0,0,...,51,2,3,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,full_active,0,0,0,16,"(10,50]",0,0,0,...,251,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,full_active,0,0,0,122,"(100,150]",4,"[1,10)",0,...,99,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7171,7171,full_active,0,0,1,52,"(50,100]",1,"[1,10)",0,...,259,1,4,0,1.0,0.0,0.0,0.0,0.0,0.0
7172,7172,full_active,0,0,0,45,"(10,50]",2,"[1,10)",2,...,11,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0
7173,7173,full_active,0,0,0,615,500+,3,"[1,10)",2,...,51,2,2,0,1.0,0.0,0.0,0.0,0.0,0.0
7174,7174,full_active,0,0,0,959,500+,0,0,0,...,107,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0


# Step 2: Feature Engineering

- Create meaningful features from interaction and metadata (e.g., content tags, user activity history)
- Build user-item interaction matrix
- Optionally extract time-based or popularity-based features

## Item category encoding

We have the caracteristics of the videos (author_id, video_type...) but this part requires less preprocessing.

For Content-based filtering, we need to use features of the videos (list of tags). No need for TF-IDF, we will use a simple one-hot encoding.

In [25]:
# Use MultiLabelBinarizer to manage efficiently the feat column
mlb = MultiLabelBinarizer()

item_features = pd.DataFrame(mlb.fit_transform(item_categories["feat"]), 
                  columns=mlb.classes_,
                  index=item_categories["video_id"]).reset_index()


In [26]:
item_features

,video_id,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,10723,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10724,10724,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10725,10725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10726,10726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Item tower

In [ ]:
item_feature_columns = [i for i in range(1, 31)]

## User tower

In [ ]:
user_feature_columns = [
    "onehot_feat9", "onehot_feat10", "onehot_feat11", 
    "onehot_feat12", "onehot_feat13", "onehot_feat14", 
    "onehot_feat15", "onehot_feat16", "onehot_feat17"
]
user_features = user_features[user_feature_columns.append("user_id")]



user_features.fillna(0, inplace=True)

# Step 3: Two-Tower Model
Two-Tower is an embedding model used mostly for retrieval tasks, such as search or recommendation.

Two towers refer to the two separate neural networks that are used to encode the user and item features. Each tower is trained independently, and the outputs of the two towers are combined to make predictions.

It is meant to be efficient for large data, and scalable to new users and items.

The model is cut into 4 parts:
- Data preparation and tuning
- Model training
- Model evaluation
- Model saving


## Model 1: Basic Two-Tower Model no features

### Data preparation

In [28]:
# Train test split

y_train, y_test = train_test_split(big_matrix.iloc[:1_000_000], test_size=0.2, random_state=42)

In [29]:
y_train = y_train.merge(user_features, on="user_id", how="left")
y_train = y_train.merge(item_features, on="video_id", how="left")

y_test = y_test.merge(user_features, on="user_id", how="left")
y_test = y_test.merge(item_features, on="video_id", how="left")

In [ ]:
# Split for each towers


user_train = y_train[user_feature_columns].to_numpy()
item_train = y_train[item_feature_columns].to_numpy()
y_train = y_train["watch_ratio"].to_numpy()

user_test = y_test[user_feature_columns].to_numpy()
item_test = y_test[item_feature_columns].to_numpy()
y_test = y_test["watch_ratio"].to_numpy()


### Model creation

In [ ]:
# Item tower

num_item_features = item_train.shape[1]

# Input layer
input_item = Input(shape=(num_item_features,), name="item_input")

# Process features
item_NN = tf.keras.models.Sequential(
    [
        Dense(256, activation="relu", name="item_x"),
        Dense(128, activation="relu", name="item_embedding"),
    ], name="item_NN"
)

item_embedding = item_NN(input_item)

In [ ]:
# User tower

num_user_features = user_train.shape[1]

# Input layer
input_user = Input(shape=(num_user_features,), name="user_input")

# Process features
user_NN = tf.keras.models.Sequential(
    [
        Dense(256, activation="relu", name="user_x"),
        Dense(128, activation="relu", name="user_embedding"),
    ], name="user_NN"
)
user_embedding = user_NN(input_user)

In [33]:
output = Dot(axes=1, name="dot_product")([user_embedding, item_embedding])

model = Model(inputs=[input_user, input_item], outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 64)        │      9,536 │ user_input[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 64)        │     12,224 │ item_input[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_product (Dot)   │ (None, 1)         │          0 │ sequential_3[0][… │
│                     │                   │            │ sequential_2[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,760 (85.00 KB)

 Trainable params: 21,760 (85.00 KB)

 Non-trainable params: 0 (0.00 B)

### Training

In [34]:
history = model.fit(
    [user_train, item_train],
    y_train,
    epochs=5,
    validation_data=([user_test, item_test], y_test),
)


Epoch 1/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 92s 4ms/step - loss: 3.2906 - root_mean_squared_error: 1.8109 - val_loss: 2.3469 - val_root_mean_squared_error: 1.5320
Epoch 2/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 90s 4ms/step - loss: 3.5713 - root_mean_squared_error: 1.8862 - val_loss: 2.3499 - val_root_mean_squared_error: 1.5330
Epoch 3/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 90s 4ms/step - loss: 3.1308 - root_mean_squared_error: 1.7681 - val_loss: 2.3519 - val_root_mean_squared_error: 1.5336
Epoch 4/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 92s 4ms/step - loss: 3.2998 - root_mean_squared_error: 1.8085 - val_loss: 2.3441 - val_root_mean_squared_error: 1.5310
Epoch 5/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 89s 4ms/step - loss: 3.2849 - root_mean_squared_error: 1.8106 - val_loss: 2.3454 - val_root_mean_squared_error: 1.5315


### Evaluation

In [36]:
# Evaluate on the test set
test_loss = model.evaluate([user_test, item_test], y_test)
print(f"Test loss: {test_loss[0]:.4f}")

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 2.3806 - root_mean_squared_error: 1.5386
Test loss: 2.3454


### Saving

In [38]:
model.save("two_tower_model.keras")

# Step 4: Two-Tower Recommendation

- Predict which videos are likely to be enjoyed by each user in the test set
- Generate a top-N ranked list of recommendations for each user

### Loading model

In [ ]:
model = tf.keras.models.load_model("two_tower_model.keras")

### Recommendation

In [ ]:
test_predictions = model.predict([test["user_id"], test["video_id"]])

In [ ]:
test_results = pd.DataFrame({
    "user_id": test["user_id"],
    "video_id": test["video_id"],
    "watch_ratio": test["watch_ratio"],
    "predicted_watch_ratio": test_predictions.flatten()
})
test_results

In [ ]:
test_results["error"] = round(abs(test_results["watch_ratio"] - test_results["predicted_watch_ratio"]), 2)
test_results.sort_values(by="error", inplace=True)

In [ ]:
test_results

# Evaluation

- Choose suitable metrics (e.g., Precision@K, Recall@K, MAP, NDCG)
- Evaluate performance and provide interpretations